# Metrical Analysis of Sanskrit Ninth Class Verb Forms

## Getting Verbal Roots 

In [ ]:
!mkdir -p downloads
!mkdir -p data

In [ ]:
!wget -O downloads/whitney_roots.pdf http://gretil.sub.uni-goettingen.de/gretil_elib/Whi885__Whitney_Roots-ACCENTED.pdf

In [4]:
# install pdftk if not already there. eg: for ubuntu: sudo apt install pdftk
!pdftk downloads/whitney_roots.pdf cat 229 output data/whitney_roots_ninth_class.pdf

In [5]:
# produces data/whitney_roots_ninth_class.txt
!pdftotext data/whitney_roots_ninth_class.pdf

Cleanup the text version manually, fixing formatting and diacritics.

Final results are in [data/whitney_roots_ninth_class_cleaned.txt](data/whitney_roots_ninth_class_cleaned.txt)

In [225]:
# TODO try to get the 9th class forms/roots directly from Lubotksy's concordance?

## Parsing Verbal Roots Info

In [1]:
CLASS_HEADER = "6. nā-class"
EARLIER_LANGUAGE_HEADER = "A. Earlier Language"
EARLIER_AND_LATER_LANGUAGE_HEADER = "B. Earlier and Later Language"
LATER_LANGUAGE_HEADER = "C. Later Language"

NINTH_CLASS_STRONG_MARKER = "ā"
NINTH_CLASS_WEAK_MARKER = "ī"

whitney_roots = []

language_period = None

with open("data/whitney_roots_ninth_class_cleaned.txt", 'r') as whitney_file:
    while line := whitney_file.readline():
        variant_no = None
        attestation_texts = None
        weak_only = False
        
        line = line.rstrip()
        if not line or CLASS_HEADER in line:
            continue    
        elif EARLIER_LANGUAGE_HEADER in line:
            language_period = "Earlier"
            continue
        elif EARLIER_AND_LATER_LANGUAGE_HEADER in line:
            language_period = "Earlier & Later"
            continue
        elif LATER_LANGUAGE_HEADER in line:
            language_period = "Later"
            continue
                
        line_parts = line.split()
        if line_parts[0].isdigit():
            variant_no = line_parts.pop(0)
        stem = line_parts.pop(0)
        if line_parts:
            attestation_texts = " ".join(line_parts)
        
        if stem.endswith(NINTH_CLASS_WEAK_MARKER):
            weak_only = True
            weak_stem = stem
            strong_stem = stem[:-1] + NINTH_CLASS_STRONG_MARKER
        else:
            weak_stem = stem[:-1] + NINTH_CLASS_WEAK_MARKER
            strong_stem = stem
        
        # removes the last two chars
        root = stem[:-2]
        # FIXME implement overrides for others
        if root == "pu":
            root = "pū"
        elif root == "ju":
            root = "jū"
        elif root == "ji":
            root = "jī"
        # FIXME detect other roots existing in variants like this
        elif root == "vr̥" and variant_no == '2':
            # variant_no 1 'cover' is attested only in AV. 2 is 'choose'
            # FIXME set stem as variant here too and ensure both are searched for
            # but long variant doesn't have any hits here so safe to ignore this here
            #root = "vr̥ ~ vr̥̄"
            root = "vr̥ vr̥̄"
        #elif root == "iṣ":
            # following vedaweb/lubotsky no 'send'(whitney has this as no 2)
            # TODO automate this logic when no hits are found for the root
        #    root += " 1"
        #elif variant_no:
            # FIXME assign vedaweb equivalent variant numbers here
        #    root += ' ' + variant_no
        # FIXME gr̥bhⁱ. or just be lax gr̥bh* would yield resulsts in lemma
        # uS attested only in weak before vowel (i.e no nI)
        # https://vedaweb.uni-koeln.de/rigveda/view/id/09.097.39
        # SrI 1 mix: yes
        # SrI 2 boil: not (VB: SrInati..)
        # aS 1 attain: not (only in mahabhharata 'aSnIs')
        # aS 2 eat yes
        # gR as gr̥̄ 1 "sing" attested 2 "swallow" not (only in AVS)
        
        whitney_roots.append({
            "root": root, 
            "variant_no": variant_no,
            "strong_stem": strong_stem,
            "weak_stem": weak_stem,
            "weak_only": weak_only,
            "attestation_texts": attestation_texts,
            "language_period": language_period,
        })

In [3]:
import pandas

In [4]:
df_whitney_roots = pandas.DataFrame.from_dict(whitney_roots)
df_whitney_roots.to_csv("data/whitney_roots_ninth_class.csv", index=None)
df_whitney_roots.head()

,root,variant_no,strong_stem,weak_stem,weak_only,attestation_texts,language_period
0,i,None,inā,inī,True,V.,Earlier
1,iṣ,None,iṣṇā,iṣṇī,False,None,Earlier
2,ubh,None,ubhnā,ubhnī,False,V.,Earlier
3,uṣ,None,uṣṇā,uṣṇī,False,V.,Earlier
4,kṣi,None,kṣiṇā,kṣiṇī,False,V.B.,Earlier


## Annotating Verbal Roots with Rig Veda Attestations (Manual)

Using Lubotsky's concordance, attestation info is manually added to [data/whitney_roots_ninth_class.csv](data/whitney_roots_ninth_class.csv).

Final results are in [data/roots_ninth_class_manual.csv](data/roots_ninth_class_manual.csv).

In [5]:
#df_roots_manual = pandas.read_csv("data/roots_ninth_class_manual.csv")
df_roots_manual = pandas.read_csv("data/roots_ninth_class_manual.csv", keep_default_na=False)

In [6]:
# TODO remove test df once we have all the annotations
#df_roots_test_manual = df_roots_manual[~df_roots["notes"].isna()]
df_roots_test_manual = df_roots_manual[df_roots_manual["notes"].str.len() > 0]
df_roots_test_manual.head()

,root,variant_no,stem,weak_only,attestation_texts,language_period,rig_veda_weak_attestations,rig_veda_strong_attestations,lubotsky_page_no,notes
1,iṣ,,iṣṇā,False,,Earlier,,1.63.2d,1:,iSnAsi
18,vr̥,1,vr̥ṇī,True,V.,Earlier,1.180.4b 1.67.1b 4.25.3a,,2:1338-1339,avRNItam vRnIte vRnIte(accented - last syll)
37,pu,,punā,False,,Earlier & Later,9.16.3c 9.67.27d,1.133.1a 10.13.3d,1:900-,punIhi puNAmi


## Annotating Verbal Roots with Rig Veda Attestations (Own Search)

### Getting Rig Veda padapatha text (Eichler)

In [ ]:
# http://www.detlef108.de/Rigveda.htm 
# http://www.detlef108.de/Notes-to-the-Rigveda-Page.htm 
!wget -O downloads/rv_padapatha_eichler.html http://www.detlef108.de/RV-Padapatha-TA3-paada-NA-UTF8.html 

In [224]:
# sudo apt install html2text
#!html2text -utf8 -width 3000 -o rv_padapatha.txt rv_padaptaha.html

from bs4 import BeautifulSoup

with open("downloads/rv_padapatha_eichler.html", "r") as input_file:
    soup = BeautifulSoup(input_file)
    
    hymns = []
    
    for para in soup.find_all("p"):
        # ignore the ending notes
        if para.contents[0].name == "span":
            continue
        
        #hymns.append(para.text.rstrip()) # no extra lines between hymns
        hymns.append(para.text)
    
    with open("data/rv_padapatha_eichler.txt", 'w') as f:
        f.write("".join(hymns))

In [222]:
# TODO break the padapatha verse into sub-lines

### Getting Rig Veda padapatha / metrically restored texts

In [ ]:
# https://github.com/cceh/c-salt_vedaweb_sources/tree/master/rigveda/versions
# description of the sources here:
# https://github.com/cceh/c-salt_vedaweb_tei/blob/master/vedaweb_corpus.tei
# https://vedaweb.uni-koeln.de/rigveda/help

!wget -O downloads/rv_padapatha_lubotsky.json https://raw.githubusercontent.com/cceh/c-salt_vedaweb_sources/master/rigveda/versions/lubotsky.json

!wget -O downloads/rv_samhitapatha_vnh.json https://raw.githubusercontent.com/cceh/c-salt_vedaweb_sources/master/rigveda/versions/vnh.json

In [217]:
# TODO make text version from the jsons, with line numbers at the beginning

### Searching text for ninth-class verbal forms

In [359]:
# TODO search text for ninth-class verbal forms , replicating vedaweb search below?
# use vidyut to identify only finite verbal forms?

## Annotating Verbal Roots with Rig Veda Attestations

In [11]:
# TODO
#!curl -H "Content-Type: application/json" -XPOST https://vedaweb.uni-koeln.de/rigveda/api/search/grammar -d '{}'

import json
import requests
import time

from bs4 import BeautifulSoup
from pprint import pprint

VEDAWEB_API_URL = "https://vedaweb.uni-koeln.de/rigveda/api"

def parse_vedaweb_search_highlight_text(text):
    word_instances = []
    
    for instance_text in BeautifulSoup(text, "lxml").text.split('/'):
        word_gloss = {}
        for prop in instance_text.split(';'):
            prop_parts = prop.split(':')
            prop_name = prop_parts[0].strip()
            prop_value = prop_parts[1].strip()
            
            #if prop_name in ["lemma", "lemma type"]:
            #    continue
                    
            word_gloss[prop_name] = prop_value
            
        word_instances.append(word_gloss)

    return word_instances
        
def search_verb_form_attestations_vedaweb_helper(root, stem=None, results_no=10, results_from=0):
    search_block = {
        "lemma type": "root",
        # vedaweb search doesn't like the ~ to mark variant roots like vr̥ ~ vr̥̄
        "lemma": root.replace(" ~ ", " ") ,
        # make sure we get verbal forms only
        # (i.e. ignore nominal forms like participles which are not marked for person)
        "person": "*", # is present
        "required": True,
        "distance": 0
    }
    
    if stem:
        search_block["term"] = '*' + stem + '*'
    
    response = requests.post(
        VEDAWEB_API_URL + "/search/grammar",
        headers = {"Content-Type": "application/json"},
        json = {
            "mode": "grammar",
            "accents": False,
            "blocks": [search_block],
            "scopes": [],
            "meta": {
                #"hymnAddressee": [],
                #"hymnGroup": [],
                #"strata": [],
                #"stanzaType": [],
                #"lateAdditions": []
            },
            "size": results_no,
            "from": results_from
        }
    )

    # raises an exception on non-200 responses, since we want to know and act on it
    response.raise_for_status()
    
    #pprint(response.request.body)
    #pprint(response.json()["hits"][0])
    
    response_json = response.json()
    
    results = {}
    found_lemmas = set()
    for hit in response_json["hits"]:
        stanza_no = hit["docId"]
        
        words = []
        for word, highlight_text in hit["highlight"].items():
            word_instances = parse_vedaweb_search_highlight_text(highlight_text) 
            for word_gloss in word_instances:
                lemma = word_gloss.pop("lemma")
                #lemma = word_gloss.get("lemma")
                # we don't need this since everything is root for us
                word_gloss.pop("lemma type")
                words.append({
                    "word": word,
                    "gloss": word_gloss
                })
                found_lemmas.add(lemma)
        
        if stanza_no in results:
            # shouldn't happen in our case at all, but just in case
            raise Exception(f"Unexpected, duplicate stanza number found: {stanza_no}")
        else:
            results[stanza_no] = words
    
    # safeguard while testing
    time.sleep(0.5)
    return {
        "lemmas": list(found_lemmas),
        "count": len(results),
        "total": response_json["total"], 
        "results": results
    }

ROOT_VARIANT_SUFFIXES = [
    # FIXME try with different order, see if we get different results?
    # also need to try all of them?
    # marker of seṭ roots (goes back to PIE roots ending in laryngeals)
    "ⁱ",
    # numbers used to mark roots of same phonetic shape but different semantics 
    " 1", " 2", 
    # FIXME try this next
    #" 3",
    # up to 3 really should be enough but just in case
    # FIXME try with just these for roots where we don't get any hits
    #" 4", " 5",
    # last resort: matches anything that follows
    # needed to catch cases like vr̥ ~ vr̥̄ in vedaweb
    # kind of safe because we also limit our searches by stem where we specify
    # the suffix directly following the root, but to limit false positives,
    # we put this last and specify predicatable variants above
    #"*"
]

def search_verb_form_attestations_vedaweb(root, stem=None, results_no=10, results_from=0):
    results = search_verb_form_attestations_vedaweb_helper(root, stem, results_no)
    
    if results["count"] == 0:
        for root_variant_suffix in ROOT_VARIANT_SUFFIXES:
            # our initial root data comes from whitney but how it's represented in vedaweb
            # may be different so gotta try with other variant forms
            root_variant = root + root_variant_suffix
            # useful while debugging
            #print(
            #    f"No results found for root:{root} stem:{stem}.",
            #    f"Retrying with root variant: {root_variant}"
            #)
            results = search_verb_form_attestations_vedaweb_helper(root_variant, stem, results_no)

            # we found a match for a working suffix so stop now
            if results["count"] > 0:
                break
            
            
    # FIXME implement getting all results    
    all_results = results["results"]    
    all_results_count = len(all_results)   
        
    # FIXME test this with root like *uS* and stem *uSn*
    # FIXME need to do this after appending number
    if len(results["lemmas"]) > 1:
        raise Exception(
            f"Unexpected, multiple roots found while searching for {root}: {results['lemmas']}"
        )
    
    return {
        "root_vedaweb": results["lemmas"][0] if (all_results_count > 0) else None,
        "count": all_results_count,
        "total": results["total"],
        "results": all_results
    }
        
# TODO need to add variant number for roots like is- (whitney 2, vedaweb 1, lubotsky 1 'to send')

roots = []

roots_attested_words_by_stanza = {}

for root in whitney_roots:   
    #root_base = "iṣ"
    #variant_no_vedaweb = "1" # TODO try up to 5 and see if we get hits still (if we don't get on base)
    #stem = "iṣṇā"
    #root_variant = root_base + ' ' + variant_no_vedaweb
    #results = search_verb_form_attestations_vedaweb(root_variant, stem, 10)
    
    # test cases
    #results = search_verb_form_attestations_vedaweb("iṣ 1", "iṣṇā", 10)
    #results = search_verb_form_attestations_vedaweb("pū", "pun", 10)

    # FIXME remove this test filter
    #if root["root"] not in ["iṣ 1", "pū", "vr̥~ vr̥̄"]:
    ##if root["root"] not in ["pū", "gr̥bh"]:
    #if root["root"] not in ["pū", "gr̥bh", "iṣ", "vr̥ vr̥̄", "vr̥"]:
        #root["strong_attestations"] = ''
        #root["weak_attestations"] = ''
        #roots.append(root)
        ##continue
        
    # FIXME automate this logic when no hits are found for the root
    #root_variant = root["root"]
    #if root_variant == "iṣ":
    #    root_variant += " 1"
    # variant_no 1 'cover' is attested only in AV. 2 is 'choose'
    #elif root_variant == "vr̥" and root["variant_no"] == '2':
    #    root_variant = "vr̥ vr̥̄"
    #pprint(root_variant)
  
    #root_variant = root["root"].replace(" ~ ", ' ')

    #root_variant = root["root"] + '*'
    
    results_strong = search_verb_form_attestations_vedaweb(
        root["root"], root["strong_stem"], 1
    )
    results_weak = search_verb_form_attestations_vedaweb(
        root["root"], root["weak_stem"], 1
    )

    #pprint(results_strong["09.067.27"])
    #pprint(results_strong)
    #pprint(results_weak)
    
    # FIXME test for this
    # unlikely to happen but still test for this
    if (results_strong["root_vedaweb"] and results_weak["root_vedaweb"] and
            results_strong["root_vedaweb"] != results_weak["root_vedaweb"]
        ):
        raise Exception(
            f"Root '{results_strong['root_vedaweb']}' inferred from strong-stem results " +
            f"does not match root '{results_weak['root_vedaweb']}' from weak-stem results"
        )
    # both strong and weak results are the same (or one of them is None) by this point
    # so can generalize in either order here  
    results_root_vedaweb = results_weak["root_vedaweb"] or results_strong["root_vedaweb"]
    
    if results_root_vedaweb and root["root"] != results_root_vedaweb:
        # FIXME add notes colummn?: root form originally present as ""
        print(
            f"note: using '{results_root_vedaweb}' for root {root['root']}",
            "(as done in vedaweb)"
        )
        # FIXME safe to override root by this point? compare final results and do so
        root["root"] = results_root_vedaweb
        #root["root_variant_vedaweb"] = results_root_vedaweb
    #else:
    #    root["root_variant_vedaweb"] = ''
    
    root["strong_attestations"] = " ".join(list(results_strong["results"].keys()))
    root["strong_attestations_total"] = results_strong['total']
    
    root["weak_attestations"] = " ".join(list(results_weak["results"].keys()))
    root["weak_attestations_total"] = results_weak['total']
    
    #root["strong_attestations_data"] = results_strong
    #root["weak_attestations_data"] = results_weak
    
    roots.append(root)
    
    # save full results data, for use later
    roots_attested_words_by_stanza[root["root"]] = {
        "strong": results_strong["results"],
        "weak": results_weak["results"]
    }
    
    # useful diagnostic message
    print(
        # print the variant root form found in vedaweb, if it's there
        #f"{' (' + root['root_variant_vedaweb'] + ')' if root['root_variant_vedaweb'] else ''}", 
        f"{root['root']}: {results_strong['total']} strong, {results_weak['total']} weak attestations"
    )
    
    # so that we don't hammer the api
    #time.sleep(0.5)

#pprint(roots)

with open(f"data/roots_ninth_class_attestations.json", 'w') as f:
    json.dump(roots_attested_words_by_stanza, f, indent=2, ensure_ascii=False)

df_roots = pandas.DataFrame.from_dict(roots)
df_roots.to_csv("data/roots_ninth_class.csv", index=None)
df_roots.head()

i: 0 strong, 0 weak attestations
note: using 'iṣ 1' for root iṣ (as done in vedaweb)
iṣ 1: 1 strong, 0 weak attestations
ubh: 2 strong, 0 weak attestations
uṣ: 0 strong, 0 weak attestations
kṣi: 0 strong, 0 weak attestations
gr̥: 0 strong, 0 weak attestations
gr̥bhⁱ: 10 strong, 6 weak attestations
jū: 4 strong, 1 weak attestations
jī: 0 strong, 0 weak attestations
dr̥: 0 strong, 0 weak attestations
drū: 0 strong, 0 weak attestations
pr̥: 0 strong, 0 weak attestations
pruṣ: 0 strong, 0 weak attestations
bhrī: 0 strong, 0 weak attestations
mī̆: 0 strong, 0 weak attestations
mr̥: 0 strong, 0 weak attestations
ram: 4 strong, 0 weak attestations
ri: 0 strong, 0 weak attestations
vr̥: 0 strong, 0 weak attestations
vlī̆: 0 strong, 0 weak attestations
śam: 0 strong, 0 weak attestations
ścam: 0 strong, 0 weak attestations
note: using 'śrathⁱ' for root śrath (as done in vedaweb)
śrathⁱ: 1 strong, 1 weak attestations
śrī: 0 strong, 5 weak attestations
śrī: 0 strong, 5 weak attestations
si: 0 stro

,root,variant_no,strong_stem,weak_stem,weak_only,attestation_texts,language_period,strong_attestations,strong_attestations_total,weak_attestations,weak_attestations_total
0,i,None,inā,inī,True,V.,Earlier,,0,,0
1,iṣ 1,None,iṣṇā,iṣṇī,False,None,Earlier,01.063.02,1,,0
2,ubh,None,ubhnā,ubhnī,False,V.,Earlier,01.063.04,2,,0
3,uṣ,None,uṣṇā,uṣṇī,False,V.,Earlier,,0,,0
4,kṣi,None,kṣiṇā,kṣiṇī,False,V.B.,Earlier,,0,,0


### Validation: Checking for missing roots

In [12]:
# roots attested in RV
df_roots.query(
    'strong_attestations_total > 0 or weak_attestations_total > 0'
)

,root,variant_no,strong_stem,weak_stem,weak_only,attestation_texts,language_period,strong_attestations,strong_attestations_total,weak_attestations,weak_attestations_total
1,iṣ 1,None,iṣṇā,iṣṇī,False,None,Earlier,01.063.02,1,,0
2,ubh,None,ubhnā,ubhnī,False,V.,Earlier,01.063.04,2,,0
6,gr̥bhⁱ,None,gr̥bhṇā,gr̥bhṇī,False,V.B.,Earlier,01.055.02,10,09.046.04,6
7,jū,None,junā,junī,False,V.,Earlier,01.027.07,4,09.079.02,1
16,ram,None,ramṇā,ramṇī,False,V.B.,Earlier,02.012.02,4,,0
22,śrathⁱ,None,śrathnā,śrathnī,False,V.,Earlier,10.171.03,1,09.069.03,1
23,śrī,1,śrīṇā,śrīṇī,False,V.B.,Earlier,,0,05.006.09,5
24,śrī,2,śrīṇā,śrīṇī,False,V.B.,Earlier,,0,05.006.09,5
31,aśⁱ,2,aśnā,aśnī,False,None,Earlier & Later,09.067.31,3,07.073.02,1
32,krī,None,krīṇā,krīṇī,False,None,Earlier & Later,04.024.10,1,,0


In [13]:
# to print without index on the left
#from IPython.display import HTML
#HTML(df_roots.to_html(index=False))

# roots not attested in RV
df_roots.query(
    'strong_attestations_total == 0 and weak_attestations_total == 0'
)

,root,variant_no,strong_stem,weak_stem,weak_only,attestation_texts,language_period,strong_attestations,strong_attestations_total,weak_attestations,weak_attestations_total
0,i,None,inā,inī,True,V.,Earlier,,0,,0
3,uṣ,None,uṣṇā,uṣṇī,False,V.,Earlier,,0,,0
4,kṣi,None,kṣiṇā,kṣiṇī,False,V.B.,Earlier,,0,,0
5,gr̥,2,gr̥ṇā,gr̥ṇī,False,V.S.,Earlier,,0,,0
8,jī,None,jinā,jinī,False,None,Earlier,,0,,0
9,dr̥,None,dr̥ṇā,dr̥ṇī,True,B.,Earlier,,0,,0
10,drū,None,drūṇā,drūṇī,False,V.B.,Earlier,,0,,0
11,pr̥,None,pr̥ṇā,pr̥ṇī,False,None,Earlier,,0,,0
12,pruṣ,None,pruṣṇā,pruṣṇī,False,B.,Earlier,,0,,0
13,bhrī,None,bhrīṇā,bhrīṇī,False,V.,Earlier,,0,,0


## Organizing Data by Verse Lines (pādas)

In [14]:
#df_roots = pandas.read_csv("data/roots_ninth_class.csv")
df_roots = pandas.read_csv("data/roots_ninth_class.csv", keep_default_na=False)
#df_roots.head()

In [17]:
# TODO remove test df once we have all the annotations
#df_roots_test = df_roots[~df_roots["rig_veda_strong_attestations"].isna()]
#df_roots_test = df_roots[df_roots["rig_veda_strong_attestations"].str.len() > 0]
df_roots_test = df_roots.query('strong_attestations != "" or weak_attestations != ""')
df_roots_test.head()

,root,variant_no,strong_stem,weak_stem,weak_only,attestation_texts,language_period,strong_attestations,strong_attestations_total,weak_attestations,weak_attestations_total
1,iṣ 1,,iṣṇā,iṣṇī,False,,Earlier,01.063.02,1,,0
2,ubh,,ubhnā,ubhnī,False,V.,Earlier,01.063.04,2,,0
6,gr̥bhⁱ,,gr̥bhṇā,gr̥bhṇī,False,V.B.,Earlier,01.055.02,10,09.046.04,6
7,jū,,junā,junī,False,V.,Earlier,01.027.07,4,09.079.02,1
16,ram,,ramṇā,ramṇī,False,V.B.,Earlier,02.012.02,4,,0


In [135]:
#from pprint import pprint

In [26]:
rv_lines = []

roots_data = df_roots_test.to_dict("records")

for root in roots_data:
    rv_weak_line_nos = root.pop("weak_attestations").split()
    rv_strong_line_nos = root.pop("strong_attestations").split()
    # FIXME just eliminate these
    root.pop("weak_attestations_total")
    root.pop("strong_attestations_total")
    
    #rv_weak_attestations_data = root.pop("rig_veda_weak_attestations_data")
    #rv_strong_attestations_data = root.pop("rig_veda_strong_attestations_data")
    
    weak_stem = root.pop("weak_stem")
    strong_stem = root.pop("strong_stem")
    
    for line_no in rv_weak_line_nos:
        rv_lines.append({"line_no": line_no, "stem": weak_stem, "stem_type": "weak"} | root)
        
    for line_no in rv_strong_line_nos: 
        rv_lines.append({"line_no": line_no, "stem": strong_stem, "stem_type": "strong"} | root)

pprint(rv_lines[0])

{'attestation_texts': '',
 'language_period': 'Earlier',
 'line_no': '01.063.02',
 'root': 'iṣ 1',
 'stem': 'iṣṇā',
 'stem_type': 'strong',
 'variant_no': '',
 'weak_only': False}


### Parsing line numbers

In [27]:
# "1.1.1b" > "01" "001" "02" "b"
# 01.063.02 > "01" "063" "02" ""
def parse_rv_line_no(string):
    line_no_parts = string.split(".")
    
    book = line_no_parts[0].zfill(2)
    hymn = line_no_parts[1].zfill(3)
    
    last_char = line_no_parts[2][-1]
    if last_char.isalpha():
        stanza = line_no_parts[2][:-1] # drop the last char
        pada = last_char
    else:
        stanza = line_no_parts[2]
        pada = ""

    stanza = stanza.zfill(2)

    return {
        "book"    : book,
        "hymn"    : f"{book}.{hymn}",
        "stanza"  : f"{book}.{hymn}.{stanza}",
        "pada"    : f"{book}.{hymn}.{stanza}.{pada}" if pada else ""
        #"pada_id" : pada or ''
    }    

rv_lines = [line | (parse_rv_line_no(line["line_no"])) for line in rv_lines]

pprint(rv_lines[0])

{'attestation_texts': '',
 'book': '01',
 'hymn': '01.063',
 'language_period': 'Earlier',
 'line_no': '01.063.02',
 'pada': '',
 'root': 'iṣ 1',
 'stanza': '01.063.02',
 'stem': 'iṣṇā',
 'stem_type': 'strong',
 'variant_no': '',
 'weak_only': False}


## Annotating Verse Lines

### Downloading annotation data

In [282]:
!mkdir -p downloads/vedaweb

In [28]:
import requests
import time

VEDAWEB_API_URL = "https://vedaweb.uni-koeln.de/rigveda/api"

rv_stanza_nos = sorted(list(set([line["stanza"] for line in rv_lines])))
#print(rv_stanza_nos)

In [91]:
for stanza_no in rv_stanza_nos:
    print(f"Getting data for stanza: {stanza_no}")
    
    # eg: https://vedaweb.uni-koeln.de/rigveda/api/document/id/0100102
    vedaweb_doc_id = stanza_no.replace('.', '')
    vedaweb_doc_url = f"{VEDAWEB_API_URL}/document/id/{vedaweb_doc_id}"
    
    response = requests.get(vedaweb_doc_url)
    # raises an exception on non-200 responses, since we want to know and act on it
    response.raise_for_status()
    
    with open(f"downloads/vedaweb/{stanza_no}.json", 'w') as f:
        f.write(response.text)
    
    # so that we don't hammer the api
    time.sleep(0.5)

Getting data for stanza: 01.027.07
Getting data for stanza: 01.055.02
Getting data for stanza: 01.063.02
Getting data for stanza: 01.063.04
Getting data for stanza: 01.093.04
Getting data for stanza: 01.093.06
Getting data for stanza: 01.131.04
Getting data for stanza: 01.133.01
Getting data for stanza: 01.139.01
Getting data for stanza: 02.012.02
Getting data for stanza: 04.024.10
Getting data for stanza: 04.057.07
Getting data for stanza: 05.006.09
Getting data for stanza: 07.007.03
Getting data for stanza: 07.073.02
Getting data for stanza: 08.023.16
Getting data for stanza: 09.046.04
Getting data for stanza: 09.067.24
Getting data for stanza: 09.067.31
Getting data for stanza: 09.069.03
Getting data for stanza: 09.079.02
Getting data for stanza: 10.171.03


### Enriching the lines with text and metrical info

In [29]:
with open(f"data/roots_ninth_class_attestations.json") as f:
    roots_attested_words_by_stanza = json.load(f)
    
#print(roots_attested_words_by_stanza)

In [30]:
import json


def pada_char_to_no(char):
    # TODO turn this into a dict
    match char:
        case 'a':
            return 0
        case 'b':
            return 1
        case 'c':
            return 2
        case 'd':
            return 3
        case 'e':
            return 4
        case 'f':
            return 5
        case 'g':
            return 6
        case _:
            raise Exception(f"Invalid pada char: {char}")
            
            
def get_stanza_words(stanza_padas):    
    stanza_words = {}
    
    for pada_data in stanza_padas:
        for word_grammar_data in pada_data["grammarData"]:
            word = word_grammar_data["form"]
            
            word_grammar_data_props = word_grammar_data["props"]
            word_position = word_grammar_data_props.pop("position", '')
            word_lemma_type = word_grammar_data_props.pop("lemma type", '')

            word_data = {
                # tracker for when we later search for the actual attested words
                "found": False, 
                "data": {
                    "pada_id": pada_data["id"],
                    # TODO test with this and later eliminate
                    #"pada_index": pada_data["index"], 
                    "pada_label": pada_data["label"],
                    "word": word,
                    "word_position_no": word_grammar_data["index"], # not-zero-indexed!
                    # TODO be careful of this, does not seem to be accurate
                    # (eg: for "punīhi" for 9.67.24 )
                    # TODO use these for checks
                    "word_position": word_position,
                    "word_lemma_type": word_lemma_type,
                    #"word_lemma": word_grammar_data["lemma"]
                    # TODO enable this later
                    #"word_props": word_grammar_data_props,
                    # TODO this not needed since all of it is contained in props
                    #"word_gloss": word_tracker_gloss
                }
            }
                
            if word in stanza_words:
                stanza_words[word].append(word_data)
            else:
                # need to use a list since the word may appear multiple times in the stanza
                stanza_words[word] = [word_data]
    
    return stanza_words
    

def get_words_by_pada(stanza_attested_words, stanza_padas, stanza_no=None):
    words_by_pada = []
    
    #pprint(stanza_attested_words)
    #pprint(stanza_padas)
    
    # transform data in stanza_padas to be amenable for searching the attested words
    stanza_words = get_stanza_words(stanza_padas)
    #pprint(stanza_words)
    
    for attested_word_data in stanza_attested_words:
        attested_word = attested_word_data["word"]
        attested_word_gloss = attested_word_data["gloss"]
        
        if attested_word in stanza_words:
            for word_instance in stanza_words[attested_word]:
                # if this word instance was already found, skip to the next one 
                if word_instance["found"]:
                    continue
                    
                word_instance_data = word_instance["data"]
                word_instance_lemma_type = word_instance_data.pop("word_lemma_type")
                
                # TODO check for lemma too?
                # TODO also ensure this is not causing us to drop valid lines
                if (word_instance_lemma_type and word_instance_lemma_type != "root"):
                    continue
                    
                if (word_instance_data["word_position"] and
                        "position" in attested_word_gloss and
                        # python "and" operator is short-circuiting so can access "position" below
                        # TODO can we trust this?
                        word_instance_data["word_position"] != attested_word_gloss["position"]
                    ):
                    continue                
                
                # TODO not needed since all of this info is already in word_instance_data
                #word_instance_data["gloss"] = attested_word_gloss
                words_by_pada.append(word_instance_data)
                
                # no need to do this in-place for python!
                word_instance["found"] = True  
                break
        else:
            # TODO handle this better? ok to let go maybe
            raise Exception(
                f"Word {attested_word} was not found in the stanza {stanza_no}: {stanza_padas}"
            )            
    
    #pprint(stanza_words)
    
    return words_by_pada            
            
      
def annotate_line(line):    
    with open(f"downloads/vedaweb/{line['stanza']}.json") as f:
        stanza = json.load(f)
        
        # TODO get pada no for each line (could be multiple) using data in: 
        # roots_attested_words_by_stanza
        # we will be ultimately returning multiple lines here sometimes
        #if line["pada"]:
        #    pada_no = pada_char_to_no(line["pada"][-1])
        #    #pada_no = pada_char_to_no(line["pada_id"])
        #else:
        
        #pada_no = 0
        
        stanza_attested_words = roots_attested_words_by_stanza[line["root"]][line["stem_type"]][line["stanza"]]
            
        words_by_pada = get_words_by_pada(stanza_attested_words, stanza["padas"], line["stanza"])
        #pprint(words_by_pada)
        
        padas = []
        
        for word in words_by_pada:
            pada = line | word
            
            # TODO rename line_no to location everywhere
            pada["line_no"] = pada["stanza"] + "." + pada["pada_id"]
            # this is not needed now
            pada.pop("pada")
            #pada["pada"] = pada["stanza"] + "." + pada["pada_id"]
            
            # TODO try out getting index from stanza info directly and see if we still
            # get the same results
            pada_no = pada_char_to_no(pada["pada_id"])
            # TODO testing remove
            #pada_no = 0

            for version in stanza["versions"]:
                if version["id"] == "version_lubotsky":
                    pada["text_padapatha"] = version["form"][pada_no]
                    break
        
            for version in stanza["versions"]:
                if version["id"] == "version_vannootenholland":
                    # TODO deal with * at the begining of the text here?
                    pada["text_samhitapatha"] = version["form"][pada_no]
                    pada["meter_scansion"] = version["metricalData"][pada_no]
                    break
    
            pada["meter_name"] = stanza["stanzaType"] or ''
        
            # TODO add these too:
            # get these info from hellewig too?
            # strata, late_additon
            #
            # pada_labels
            # 
            # hymn_absolute_no, hymn_addressee, hymn_group
            #
            # present in search results too: 'hymnAddressee', 'hymnGroup', 'stanzaStrata'
            
            padas.append(pada)

        return padas

    
#line_annotated = annotate_line(rv_lines[1])
#pprint(line_annotated)

#rv_lines_annotated = [annotate_line(line) for line in rv_lines]

rv_lines_annotated = []
for line in rv_lines:
    # TODO rename the annotate function here
    rv_lines_annotated.extend(annotate_line(line))

pprint(rv_lines_annotated[0])
print(f"\nTotal number of lines: {len(rv_lines_annotated)}")

{'attestation_texts': '',
 'book': '01',
 'hymn': '01.063',
 'language_period': 'Earlier',
 'line_no': '01.063.02.d',
 'meter_name': 'Triṣṭubh',
 'meter_scansion': 'SS LLS SSLS LL',
 'pada_id': 'd',
 'pada_label': 'M',
 'root': 'iṣ 1',
 'stanza': '01.063.02',
 'stem': 'iṣṇā',
 'stem_type': 'strong',
 'text_padapatha': 'púraḥ iṣṇā́si= puruhūta pūrvī́ḥ',
 'text_samhitapatha': 'púra iṣṇā́si puruhūta pūrvī́ḥ',
 'variant_no': '',
 'weak_only': False,
 'word': 'iṣṇā́si',
 'word_position': 'intermediate',
 'word_position_no': 2}

Total number of lines: 27


### Saving the Final Line Results

In [31]:
df_rv_lines = pandas.DataFrame.from_dict(rv_lines_annotated)
df_rv_lines.to_csv("data/rv_lines_ninth_class.csv", index=None)
df_rv_lines.head(100)

,line_no,stem,stem_type,root,variant_no,weak_only,attestation_texts,language_period,book,hymn,stanza,pada_id,pada_label,word,word_position_no,word_position,text_padapatha,text_samhitapatha,meter_scansion,meter_name
0,01.063.02.d,iṣṇā,strong,iṣ 1,,False,,Earlier,01,01.063,01.063.02,d,M,iṣṇā́si,2,intermediate,púraḥ iṣṇā́si= puruhūta pūrvī́ḥ,púra iṣṇā́si puruhūta pūrvī́ḥ,SS LLS SSLS LL,Triṣṭubh
1,01.063.04.b,ubhnā,strong,ubh,,False,V.,Earlier,01,01.063,01.063.04,b,Mh,ubhnā́ḥ,5,,vr̥trám yát vajrin= vr̥ṣakarman ubhnā́ḥ,vr̥tráṁ yád vajrin vr̥ṣakarman ubhnā́ḥ,LL L LL SSLS LL,Triṣṭubh
2,09.046.04.b,gr̥bhṇī,weak,gr̥bhⁱ,,False,V.B.,Earlier,09,09.046,09.046.04,b,MO,gr̥bhṇīta,2,verse final,śukrā́ gr̥bhṇīta manthínā,śukrā́ gr̥bhṇīta manthínā,LL LLS LSL,Gāyatrī
3,01.055.02.b,gr̥bhṇā,strong,gr̥bhⁱ,,False,V.B.,Earlier,01,01.055,01.055.02,b,M,gr̥bhṇāti,2,intermediate,práti gr̥bhṇāti= víśritāḥ} várīmabhiḥ,práti gr̥bhṇāti víśritā várīmabhiḥ,SS LLS LSL SLSL,Jagatī
4,09.079.02.b,junī,weak,jū,,False,V.,Earlier,09,09.079,09.079.02,b,M,junīmási,5,,dhánā vā yébhiḥ= árvataḥ} junīmási,dhánā vā yébhir árvato junīmási,SL L LS LSL SLSS,Jagatī
5,01.027.07.b,junā,strong,jū,,False,V.,Earlier,01,01.027,01.027.07,b,M,junā́ḥ,4,,ávāḥ vā́jeṣu yám junā́ḥ,ávā vā́jeṣu yáṁ junā́ḥ,SL LLS L SL,Gāyatrī
6,02.012.02.b,ramṇā,strong,ram,,False,V.B.,Earlier,02,02.012,02.012.02,b,M,áramṇāt,4,,yáḥ párvatān= prákupitān} áramṇāt,yáḥ párvatān prákupitām̐ áramṇāt,L LSL SSSL SLL,Triṣṭubh
7,09.069.03.b,śrathnī,weak,śrathⁱ,,False,V.,Earlier,09,09.069,09.069.03,b,M,śrathnīté,1,intermediate,śrathnīté?_ naptī́ḥ= áditeḥ} r̥tám yaté-_,śrathnīté naptī́r áditer r̥táṁ yaté,LLL LL SSL SL SL,Jagatī
8,10.171.03.c,śrathnā,strong,śrathⁱ,,False,V.,Earlier,10,10.171,10.171.03,c,M,śrathnāḥ,2,verse final,múhuḥ śrathnāḥ manasyáve?_,múhuḥ śrathnā manasyáve,SL LL SLSL,Gāyatrī
9,05.006.09.b,śrīṇī,weak,śrī,1,False,V.B.,Earlier,05,05.006,05.006.09,b,M,śrīṇīṣe,2,verse final,dárvī+_ śrīṇīṣe?_ āsáni,dárvī śrīṇīṣa āsáni,LL LLS LSS,


## Validating the data

### Checking for missing roots

In [33]:
import numpy as np

In [244]:
# our starting list of roots
#print(np.sort(df_roots["root"].unique()))

Present roots:

In [34]:
print(np.sort(df_rv_lines["root"].unique()))

['aśⁱ' 'gr̥bhⁱ' 'gr̥hⁱ' 'iṣ 1' 'jū' 'krī' 'mathⁱ' 'muṣⁱ' 'prī' 'pū' 'ram'
 'ubh' 'vr̥ vr̥̄' 'śrathⁱ' 'śrī']


Missing roots:

In [35]:
print(np.setdiff1d(df_roots["root"].unique(), df_rv_lines["root"].unique()))

['badh' 'bhrī' 'dhū̆' 'drū' 'dr̥' 'grath' 'gr̥' 'hru' 'hr̥' 'i' 'jā' 'jī'
 'kliś' 'kuṣ' 'kṣi' 'lu' 'mr̥' 'mr̥d' 'mī̆' 'pruṣ' 'pr̥' 'puṣ' 'ri' 'si'
 'skabh' 'spr̥' 'stabh' 'str̥' 'subh' 'uṣ' 'vlī̆' 'vr̥' 'śam' 'ścam' 'śr̥']


### Checking found word against padapatha text

In [36]:
mismatches = []

no_of_mismatches = 0

LUBOTSKY_PADAPATHA_SPECIAL_CHARACTERS = ['-', '_', '=', '?', '+']

def clean_lubotsky_padapatha(string):
    return ''.join(c for c in string if c not in LUBOTSKY_PADAPATHA_SPECIAL_CHARACTERS)


for line in rv_lines_annotated:
    padapatha_text_cleaned = clean_lubotsky_padapatha(line["text_padapatha"])
    padapatha_parts = padapatha_text_cleaned.split(' ')
    
    word_from_padapatha = padapatha_parts[line["word_position_no"] - 1]
    
    if line["word"] != word_from_padapatha:
        no_of_mismatches += 1
        print(
            f"{line['line_no']}:{line['word_position_no']}", 
            f"({line['stem']}) {line['word']} ≠ {word_from_padapatha}",
            #f"[{padapatha_text_cleaned}]",
            f"[{line['text_padapatha']}]"
        )
        
print(f"\nFound {no_of_mismatches} mismatches.")


Found 0 mismatches.


Ignore cases of final vowel lengthening above (especially with imperatives) -- they are not really mismatches.